In [1]:
import numpy as np
import pandas as pd

In [2]:
# Reading the database through pandas library

In [3]:
movie_data = pd.read_csv("tmdb_5000_movies.csv")
movie_credit = pd.read_csv("tmdb_5000_credits.csv")

# Merging both the dataset
complete_data = movie_data.merge(movie_credit,on="title")

In [4]:
# Getting the data useful for the project

In [5]:
# movie_data = movie_data[['genres','id','keywords','overview']]
# movie_credit = movie_credit[['cast','crew']]
complete_data = complete_data[['genres','title','movie_id','keywords','overview','cast','crew']]
# complete_data.head()

In [6]:
# Dropping NA values 
complete_data.dropna(inplace=True)
# Removing null entries
complete_data.isnull().sum()

genres      0
title       0
movie_id    0
keywords    0
overview    0
cast        0
crew        0
dtype: int64

In [7]:
# complete_data.iloc[0].genres

In [8]:
# Importing useful modules and libraries

import ast
import sklearn
import re
import nltk
# nltk.download('punkt')
# nltk.download('stopwords')

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [9]:
ps = PorterStemmer()
tfidf = TfidfVectorizer()

In [10]:
# Functions for converting the data into list as we cannot operate much with objects and pulling out
# the important data and discarding which we dont need

def convert(obj):
    l = []
    for i in ast.literal_eval(obj):
        l.append(i['name'])
    return l

def convertcast(obj):
    l = []
    for i in ast.literal_eval(obj):
        l.append(i['name'])
        if(len(l) == 3):
            break
    return l

def director(obj):
    l = []
    for i in ast.literal_eval(obj):
        if(i['job']=='Director'):
            l.append(i['name'])
    return l

def remove_space(obj):
    l=[]
    for s in obj:
        x="".join(s.split(" "))
        l.append(x)
    return l

In [11]:
complete_data['genres']=complete_data['genres'].apply(convert)
complete_data['keywords']=complete_data['keywords'].apply(convert)
complete_data['cast']=complete_data['cast'].apply(convertcast)
complete_data['crew']=complete_data['crew'].apply(director)
complete_data['overview']=complete_data['overview'].apply(lambda x:x.split())

# remove_space(complete_data['cast'])
# complete_data['cast'] = complete_data['cast'].apply(remove_space)
# complete_data['genres'] = complete_data['genres'].apply(remove_space)
# complete_data['crew'] = complete_data['crew'].apply(remove_space)
# complete_data['keywords'] = complete_data['keywords'].apply(remove_space)
# Applying DRY Principle
for types in ['cast', 'genres', 'crew', 'keywords']:
    complete_data[types] = complete_data[types].apply(remove_space)

In [12]:
# Merging all to create a single column called tags.

complete_data['tags']=complete_data['overview']+complete_data['genres']+complete_data['keywords']+complete_data['cast']+complete_data['crew']

In [13]:
# complete_data.head()

In [14]:
# Creating new dataframe

new_df = complete_data[['movie_id','title','tags']]

In [15]:
new_df['tags']=new_df['tags'].apply(lambda x:" ".join(x))

C:\Users\anushtha bageria\AppData\Local\Temp\ipykernel_31256\487797088.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags']=new_df['tags'].apply(lambda x:" ".join(x))


In [16]:
# new_df['tags']

In [17]:
# Performing stemming

def stem(text):
    y=[]
    
    for i in text.split():
        y.append(ps.stem(i))
    return " ".join(y)

In [18]:
new_df['tags']=new_df['tags'].apply(stem)
new_df['tags']=new_df['tags'].str.lower()

C:\Users\anushtha bageria\AppData\Local\Temp\ipykernel_31256\1429525103.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags']=new_df['tags'].apply(stem)
C:\Users\anushtha bageria\AppData\Local\Temp\ipykernel_31256\1429525103.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags']=new_df['tags'].str.lower()


In [19]:
# bag of words algo using library

In [20]:
# from sklearn.feature_extraction.text import CountVectorizer
# cv = CountVectorizer(max_features=5000,stop_words='english')

In [21]:
# X=cv.fit_transform(new_df['tags']).toarray()

In [22]:
# bag of words algo without using library

In [23]:
# word2count= {}
# stop_words = set(stopwords.words('english'))
# print(stop_words)

In [24]:
# def vectorization(sentence):
#     words = nltk.word_tokenize(sentence)
#     for word in words:
#         if word not in word2count.keys():
#             word2count[word] = 1
#         else:
#             word2count[word] += 1

# for data in new_df['tags']:
#     words = nltk.word_tokenize(data)
#     filtered_sentence = [w for w in words if not w.lower() in stop_words]
#     for word in filtered_sentence:
#         if word not in word2count.keys():
#             word2count[word] = 1
#         else:
#             word2count[word] += 1

In [25]:
# new_df['tags'].apply(vectorization)

In [26]:
# print(word2count)

In [27]:
# import heapq
# freq_words = heapq.nlargest(100, word2count, key=word2count.get)

In [28]:
# X = []
# for data in new_df['tags']:
#     vector = []
#     for word in freq_words:
#         if word in nltk.word_tokenize(data):
#             vector.append(1)
#         else:
#             vector.append(0)
#     X.append(vector)
# X = np.asarray(X)

In [29]:
# print(X)

In [30]:
# Using tf-idf for vectorization

In [31]:
# bagOfWords = []
# for i in new_df['tags']:
#     bagOfWords.append(i.split(' '))

In [32]:
# uniqueWords = set()
# wordsList = []
# for i in new_df['tags']:
#     for word in i.split(' '):
#         wordsList.append(word)
# uniqueWords = set(wordsList)

In [33]:
# word2Count = []
# for data in new_df['tags']:
#     wordsDict = dict.fromkeys(uniqueWords, 0)
#     for word in data.split(' '):
#         wordsDict[word]+=1
#     word2Count.append(wordsDict)

In [34]:
# def computeTF(wordDict, bagOfWordsList):
#     tfDict = {}
#     bagOfWordsCount = len(bagOfWordsList)
#     for word, count in wordDict.items():
#         tfDict[word] = count / float(bagOfWordsCount)
#     return tfDict

In [35]:
# tfList = []
# j = 0
# for i in new_df['tags']:
#     tfList.append(computeTF(word2Count[j], i.split(' ')))
#     j += 1
# print(tfList)

In [36]:
# TF-IDF = Term Frequency (TF) * Inverse Document Frequency (IDF)

In [37]:
# Facing memory error so applying tf-idf using the libraries itself

In [38]:
data = []
for i in new_df['tags']:
    data.append(i)
result = tfidf.fit_transform(data)

resultMatrix = result.toarray()
print(resultMatrix)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [39]:
# for ele1, ele2 in zip(tfidf.get_feature_names(), tfidf.idf_):
#     print(ele1, ':', ele2)

In [40]:
# Using cosine similarity for finding the nearest similar movies

In [41]:
similarity = cosine_similarity(resultMatrix)

In [42]:
def recommend(movie):
    index = new_df[new_df['title'] == movie].index[0]
    distances = sorted(list(enumerate(similarity[index])),reverse=True,key = lambda x: x[1])
    
    for i in distances[1:11]:
        print(new_df.iloc[i[0]].title)

In [43]:
recommend('Avatar')

Aliens
Falcon Rising
Battle: Los Angeles
Apollo 18
Aliens vs Predator: Requiem
Meet Dave
Titan A.E.
Star Trek Into Darkness
The Book of Life
Predators


In [44]:
import pickle

In [45]:
pickle.dump(new_df.to_dict(),open('D:/movie recommendation system/recommendation system code/movies_dict.pkl','wb'))

In [46]:
pickle.dump(similarity,open('D:/movie recommendation system/recommendation system code/similarity.pkl','wb'))